In [7]:
!tune download google/gemma-2-2b-it\
    --hf-token <hf token >

/opt/conda/envs/py_3.12/lib/python3.12/site-packages/redis/connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
  warnings.warn(msg)
Ignoring files matching the following patterns: None
.gitattributes: 100%|██████████████████████| 1.57k/1.57k [00:00<00:00, 22.1MB/s]
README.md: 100%|████████████████████████████| 29.1k/29.1k [00:00<00:00, 165MB/s]
generation_config.json: 100%|██████████████████| 187/187 [00:00<00:00, 1.19MB/s]
model-00001-of-00002.safetensors:  48%|█▉  | 2.42G/4.99G [00:02<00:02, 1.27GB/s]/opt/conda/envs/py_3.12/lib/python3.12/site-packages/redis/connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
  warnings.warn(msg)
model-00001-of-00002.safetensors: 100%|████| 4.99G/4.99G [00:03<00:00, 1.62GB/s]
model-00002-of-00002.safetensors: 100%|███████| 241M/241M [00:00<00:00, 384MB/s]
model.safetensors.index.json: 100%|█████████| 24.2k/24.2k [00:00<00:00, 190MB/s]
special_tokens_map.json: 100%|███████

In [2]:
!tune ls

/opt/conda/envs/py_3.12/lib/python3.12/site-packages/redis/connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
  warnings.warn(msg)
RECIPE                                   CONFIG                                  
full_finetune_single_device              llama2/7B_full_low_memory               
                                         code_llama2/7B_full_low_memory          
                                         llama3/8B_full_single_device            
                                         llama3_1/8B_full_single_device          
                                         llama3_2/1B_full_single_device          
                                         llama3_2/3B_full_single_device          
                                         mistral/7B_full_low_memory              
                                         phi3/mini_full_low_memory               
                                         phi4/14B_full_low_memory                
        

In [3]:
!tune cp  gemma2/2B_lora_single_device gemma2bconfig.yaml

/opt/conda/envs/py_3.12/lib/python3.12/site-packages/redis/connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
  warnings.warn(msg)
Copied file to gemma2bconfig.yaml


In [15]:
!tune run lora_finetune_single_device --config gemma2/2B_lora_single_device batch_size=10 \
    checkpointer.checkpoint_dir=/tmp/gemma-2-2b-it \
    checkpointer.checkpoint_files="[model-00001-of-00002.safetensors,model-00002-of-00002.safetensors]" \
    tokenizer.path=/tmp/gemma-2-2b-it/tokenizer.model \
    checkpointer.output_dir=/tmp/tune/gemma-2-2b-it \
    dataset.source=json \
    dataset.data_files=data/database.json\
    dataset.split=train \
    dataset.train_on_input=True \
    dataset.column_map.input=input \
    dataset.column_map.output=output \
    epochs=100 \
    model.lora_rank=256 \
    model.lora_alpha=512 \


/opt/conda/envs/py_3.12/lib/python3.12/site-packages/redis/connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
  warnings.warn(msg)
INFO:torchtune.utils._logging:Running LoRAFinetuneRecipeSingleDevice with resolved config:

batch_size: 15
checkpointer:
  _component_: torchtune.training.FullModelHFCheckpointer
  checkpoint_dir: /tmp/gemma-2-2b-it
  checkpoint_files:
  - model-00001-of-00002.safetensors
  - model-00002-of-00002.safetensors
  model_type: GEMMA2
  output_dir: /tmp/tune/gemma-2-2b-it
  recipe_checkpoint: null
clip_grad_norm: null
compile: false
dataset:
  _component_: torchtune.datasets.alpaca_dataset
  column_map:
    input: input
    output: output
  data_files: data/database.json
  packed: false
  source: json
  split: train
  train_on_input: true
device: cuda
dtype: bf16
enable_activation_checkpointing: true
enable_activation_offloading: false
epochs: 80
gradient_accumulation_steps: 8
log_every_n_steps: 1
log_peak_memory_stats: tr

In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer

#TODO: update it to your chosen epoch
trained_model_path = "/tmp/tune/gemma-2-2b-it/epoch_79"

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=trained_model_path,
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(trained_model_path, safetensors=True)


# Function to generate text
def generate_text(model, tokenizer, prompt, max_length=1000):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=max_length)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)



/opt/conda/envs/py_3.12/lib/python3.12/site-packages/redis/connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
  warnings.warn(msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [20]:

prompt = "You are a database agent. Based on the user's question, identify the database tables required to find the answer. Do not write the SQL query. Just list the tables, \
query : Find all students majoring in 'Computer Science' who received a grade of 'A' in any course during the 'Fall 2024' semester"
print("Base model output:", generate_text(model, tokenizer, prompt))

Base model output: You are a database agent. Based on the user's question, identify the database tables required to find the answer. Do not write the SQL query. Just list the tables, query : Find all students majoring in 'Computer Science' who received a grade of 'A' in any course during the 'Fall 2024' semester.

**Tables:**

* Students
* Courses
* Grades

**Explanation:**

To answer the question, we need to combine information from three tables:

* **Students:** To get the student IDs.
* **Courses:** To get the course IDs and names.
* **Grades:** To get the grade and course IDs. 

